# Gerenciamento de Perfil

_Script_ que demonstra a utilização para acesso ao "Gerenciamento de Perfil" do e-SAJ.


In [1]:
import os
from pathlib import Path

from dotenv import load_dotenv

import pyesaj.scraper as esaj

<br>

---

## Credenciais

Obtemos as credenciais de um arquivo `.env` com o seguinte conteúdo:

````shell
USERNAME_TJSP=***.***.***-**
PASSWORD_TJSP=****************
````


In [ ]:
load_dotenv()

USERNAME = os.getenv('USERNAME_TJSP')
PASSWORD = os.getenv('PASSWORD_TJSP')

<br>

---

## *Driver*

Instanciamos o _driver_/navegador que será usado para manejar o e-SAJ.


In [ ]:
driver = esaj.webdriver.Chrome(headless=False)

<br>

---

## *Login*


Em 24.03.2025 o TJSP implantou a autenticação por dois fatores e, ao tentar o acesso com _login_/senha é disparado um e-mail contendo o código númerico para fazer a autenticação no sistema.

A autenticação em duas etapas (ou autenticação de duplo fator) promove a segurança do sistema e dificulta que automações sejam feitas.



In [ ]:
log = esaj.page.Login(driver=driver)

<br>

Na primeira etapa apenas entramos com os dados básicos: *login* e senha.

In [ ]:
log.login_1_etapa(username=USERNAME, password=PASSWORD)

<br>

Ao executar a primeira etapa, será disparado um e-mail com um código numérico, de 6 dígitos, para concluir a autenticação.

In [ ]:
log.login_2_etapa(token=727398)

<br>

---

## Gerenciamento de Perfis

O acesso ao "Gerenciamento de Perfil" se deve a necessidade de avaliar se o usuário tem permissão de fazer determinadas coisas no sistema e-SAJ. Para a atividade de especialização de intimações, por exemplo, o usuário deve ter perfil de  "Distribuidor Intimações MP - Segunda Instância". Dessa forma, acessando os perfis, é possível avaliar se o usuário tem, ou não, permissão de acesso.

In [ ]:
esaj_perfil = esaj.page.GerenciamentoPerfis(driver=driver)

In [ ]:
perfis = esaj_perfil.obtem_perfis()
perfis

<br>

Abaixo um código para avaliar se o usuário está habilitado ou não a um determinado perfil.

    FAZER: Encapsular a função abaixo dentro de uma Classe.


In [ ]:
check_2_instancia = [
    x.perfil
    for x in perfis
    if x.autorizado is True
    and x.perfil == 'Distribuidor Intimações MP - Segunda Instância'
]
if len(check_2_instancia) == 1:
    print(
        f'Prezado {log.user},\n\n'
        f'Você tem perfil de acesso como "Distribuidor Intimações MP - Segunda Instância" necessário para distribuir e/ou especializar as intimações.\n\n'
        f'Podemos dar sequencia!!'
    )


else:
    perfis_autorizados = [x.perfil for x in perfis if x.autorizado is True]
    perfis_autorizados = '\n'.join([f'- {x}' for x in perfis_autorizados])
    raise Exception(
        f'Prezado {log.user},\n\n'
        f'Ao que tudo indica, você não tem perfil de acesso como "Distribuidor Intimações MP - Segunda Instância", necessário para distribuir e/ou especializar as intimações.\n\n'
        f'Os perfis autorizados para você são:\n{perfis_autorizados}'
    )

<br>

Por fim, fecho o *driver*/navegador.

In [ ]:
driver.quit()